Script to calculate the equivalent GloFAS thresholds compared to the observed thresholds. Uses GloFAS reanalysis data from 1979. Uses quantile mapping approach to map values between the observed and reanalysis dataset. 

In [2]:
# import relevant packages
import numpy as np
import pandas as pd
from scipy import stats
import os

In [3]:
# define country and directory
country = 'zimbabwe'
directory = '/s3/scratch/jamie.towner/flood_aa'

output_directory = os.path.join(directory, country, "outputs/thresholds")
os.makedirs(output_directory, exist_ok=True)  # create directory if it does not already exist 

In [4]:
# define paths to data
metadata_directory = os.path.join(directory, country, "data/metadata")
observed_data_directory = os.path.join(directory, country, "data/observations/gauging_stations/all_stations")
reanalysis_data_directory = os.path.join(directory, country, "data/forecasts/glofas_reanalysis/all_stations")

observed_data_file = "observations.csv"
reanalysis_data_file = "glofas_reanalysis_zim.csv"
station_info_file = "metadata_observations.csv"

# load data
observed_data_path = os.path.join(observed_data_directory, observed_data_file)
reanalysis_data_path = os.path.join(reanalysis_data_directory, reanalysis_data_file)
station_info_path = os.path.join(metadata_directory, station_info_file)

observed_data = pd.read_csv(observed_data_path)
reanalysis_data = pd.read_csv(reanalysis_data_path)
station_info = pd.read_csv(station_info_path)

In [5]:
# convert date columns to datetime
observed_data["date"] = pd.to_datetime(observed_data["date"], format='mixed')
reanalysis_data["date"] = pd.to_datetime(reanalysis_data["date"], format='mixed')
station_info['obs_bankfull'] = pd.to_numeric(station_info['obs_bankfull'], errors='coerce')
station_info['obs_moderate'] = pd.to_numeric(station_info['obs_moderate'], errors='coerce')
station_info['obs_severe'] = pd.to_numeric(station_info['obs_severe'], errors='coerce')

In [6]:
# Remove leading/trailing whitespace from metadata station names
station_info['station name'] = station_info['station name'].str.strip()

# Remove whitespace from observed and reanalysis data columns
observed_data.columns = observed_data.columns.str.strip()
reanalysis_data.columns = reanalysis_data.columns.str.strip()

In [12]:
# initialize list to store results
results = []

# loop over each station and threshold in metadata
for index, row in station_info.iterrows():
    station = row['station name']
    
        # skip station if any threshold is missing (NaN)
    if pd.isna(row['obs_bankfull']) or pd.isna(row['obs_moderate']) or pd.isna(row['obs_severe']):
        continue
    
    # get observed and reanalysis data for the station
    data_observed = observed_data[station].dropna().values
    data_reanalysis = reanalysis_data[station].dropna().values

    # standardize both datasets (z-score normalization)
    obs_mean, obs_std = np.mean(data_observed), np.std(data_observed)
    reanalysis_mean, reanalysis_std = np.mean(data_reanalysis), np.std(data_reanalysis)

    z_observed = (data_observed - obs_mean) / obs_std
    z_reanalysis = (data_reanalysis - reanalysis_mean) / reanalysis_std

    # define thresholds to loop over
    thresholds = {
        'obs_bankfull': row['obs_bankfull'],
        'obs_moderate': row['obs_moderate'],
        'obs_severe': row['obs_severe']
    }

    # loop over each threshold
    for threshold_name, threshold_value in thresholds.items():
        # convert threshold to z-score in observed data space
        z_threshold = (threshold_value - obs_mean) / obs_std

        # get percentile rank of threshold in observed data
        percentile_rank_observed = stats.percentileofscore(z_observed, z_threshold)

        # ensure percentiles are within valid range
        percentile_rank_observed = max(0, min(percentile_rank_observed, 100))

        # interpolate the corresponding value in reanalysis data
        percentiles = np.linspace(0, 100, len(z_reanalysis))
        z_mapped = np.interp(percentile_rank_observed, percentiles, np.sort(z_reanalysis))

        # convert back to the original reanalysis scale
        value_reanalysis = (z_mapped * reanalysis_std) + reanalysis_mean

        # store results
        results.append({
            'station': station,
            'threshold_name': threshold_name,
            'threshold_value': threshold_value,
            'percentile_rank_observed': percentile_rank_observed,
            'value_reanalysis': value_reanalysis
        })

# convert results to a dataframe and print
results_df = pd.DataFrame(results)
results_df

,station,threshold_name,threshold_value,percentile_rank_observed,value_reanalysis
0,beitbridge,obs_bankfull,951.0,95.870044,232.403944
1,beitbridge,obs_moderate,2425.0,99.889868,2107.505713
2,beitbridge,obs_severe,2755.0,99.944934,2462.066664
3,makwe,obs_bankfull,12.0,99.565369,62.928758
4,makwe,obs_moderate,18.0,99.756606,75.781478
5,makwe,obs_severe,30.0,99.965229,160.242090
6,ingwesi,obs_bankfull,0.0,48.705328,0.000000
7,ingwesi,obs_moderate,0.0,48.705328,0.000000
8,ingwesi,obs_severe,3.0,99.807404,15.917023
9,manyuchi,obs_bankfull,0.7,95.095183,40.341399


In [14]:
# save output as a csv 
results_df.to_csv(os.path.join(output_directory, "glofas_return_periods.csv"), index=False)